In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
from sklearn.metrics import accuracy_score
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
Transformerslogger = logging.getLogger("transformers")
Transformerslogger.setLevel(logging.WARNING)

In [3]:
file_path = '/Users/aaq1bm/AaqibMirzaTests/financialData.csv'
review = pd.read_csv(file_path, encoding='latin1')

In [4]:
train_df = pd.DataFrame(review)

In [5]:
sentiment = { 'positive': 2, 'neutral': 1, 'negative': 0 }

In [6]:
train_df['labels'] = train_df['Sentiments'].map(sentiment)

In [7]:
train_df = train_df[["Reviews", "labels"]]

In [8]:
train_df

,Reviews,labels
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2
...,...,...
4841,LONDON MarketWatch -- Share prices ended lower...,0
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,Net sales of the Paper segment decreased to EU...,0


In [9]:
eval_df = train_df.sample(n=500, random_state=15)

In [10]:
eval_df 

,Reviews,labels
1695,We went to the market with yield guidance of t...,1
3412,There will be return flights from Stuttgart ev...,1
976,"The company , which makes garden tools , sciss...",2
921,We are now in a position to pursue novel clini...,2
534,We have made long-term investments in developi...,1
...,...,...
721,"Earnings per share EPS amounted to EUR0 .03 , ...",2
3291,The new chain has 700 discount stores and 250 ...,1
3321,The plant will collect raw material from the B...,1
1675,Thus the group 's balance sheet will have abou...,1


In [11]:
output_dir_model3 = "/Users/aaq1bm/AaqibMirzaTests/outputs3"

In [12]:
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=3,
    use_cuda=False,
    args={'output_dir': output_dir_model3}
) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def compute_metrics(self, preds, model_outputs, labels, eval_examples, **kwargs):
    result = {}
    accuracy = accuracy_score(labels, preds) 
    result['accuracy'] = accuracy
    wrong_predictions = [i for i, (pred, label) in enumerate(zip(preds, labels)) if pred != label]
    return result, wrong_predictions

In [14]:
import os

In [15]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [16]:
model.compute_metrics = compute_metrics.__get__(model)

In [17]:
train_df = train_df.drop(eval_df.index)

In [ ]:
model.train_model(train_df)

/Users/aaq1bm/anaconda3/envs/FYP/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/8 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_3_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/544 [00:00<?, ?it/s]

In [38]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/Users/aaq1bm/anaconda3/envs/FYP/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_2


Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'accuracy': 0.868, 'eval_loss': 0.34133308231830595}


In [25]:
print("wrong predictions :")
print(wrong_predictions)

wrong predictions :
[4, 7, 27, 42, 50, 56, 58, 59, 61, 62, 67, 72, 75, 80, 93, 97, 112, 117, 118, 120, 128, 133, 145, 148, 150, 156, 163, 172, 183, 186, 197, 204, 205, 226, 239, 242, 258, 267, 273, 285, 294, 297, 300, 313, 318, 359, 366, 367, 369, 372, 382, 388, 407, 419, 421, 422, 424, 426, 471, 475, 480, 482, 485, 492, 498, 499]


In [26]:
print(" result :")
print(result)

 result :
{'accuracy': 0.868, 'eval_loss': 0.34133308231830595}


In [27]:
print(" model outputs :")
print(model_outputs)

 model outputs :
[[-2.35603809  1.34725404  2.69514704]
 [-2.12895346  3.48670816 -0.30611518]
 [-2.64260077  2.61019826  1.12133455]
 ...
 [-2.65576148  3.42244267  0.35983321]
 [-2.59961414  2.36281133  1.8001858 ]
 [-2.80997062  2.20402169  1.67264199]]
